In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import pyarrow
import os
import sys
import json
import urllib.request
from google.cloud import bigquery
from google.cloud.bigquery import job
from google.oauth2 import service_account
import google.auth
import copy
PROJCECT = 'ballosodeuk'


bq = bigquery.Client(project=PROJCECT)
dataset_id = 'airbridge_mart'
table_id = 'web_keyfeatures'
table_ref = f'ballosodeuk.{dataset_id}.{table_id}'

### 기존 테이블 받아오기 어ㅉㅃ꺼ㅜ

In [ ]:

client = bigquery.Client()
query = """SELECT 
  *
FROM 
  `ballosodeuk.temp.inl_0614-0715`
"""

job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

df = query_job.to_dataframe()

In [20]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

# JSON 키 파일 경로 설정
json_keyfile_path = "C:/Users/Owner/Downloads/json/ballosodeuk-4811d21b3452.json"

# 서비스 계정 자격 증명 설정
credentials = service_account.Credentials.from_service_account_file(
    json_keyfile_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform", "https://www.googleapis.com/auth/drive"]
)

# 빅쿼리 클라이언트 생성
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# 쿼리 작성
query = """
SELECT 
  *
FROM 
  `ballosodeuk.temp.inl_0614-0715`
"""

# 쿼리 실행
query_job = client.query(query)

# 결과 불러오기
results = query_job.result()

# 결과를 데이터프레임으로 변환
df = results.to_dataframe()

# 데이터 출력
print(df)


Forbidden: 403 Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.

Location: asia-northeast3
Job ID: 6dc87e5b-7c1e-4f66-89f2-39a8021ba734


### 샘플데이터

In [3]:
df = pd.read_csv('C:\FTC_downloads\code\git\project_hct/result/cp/dynamic/results/dynamic_20240501_20240617_cleanedAndMerged.csv')
df = df.head()
product_col = 'productName'

In [36]:
#엑세스 수집
access = pd.read_csv("../../cp_data.csv")


In [39]:
media = 'nv_api'
id_1 = access.query('media=="nv_api1"')['id'].values[0]
sc_1 = access.query('media=="nv_api1"')['secret'].values[0]
id_2 = access.query('media=="nv_api2"')['id'].values[0]
sc_2 = access.query('media=="nv_api2"')['secret'].values[0]
id_3 = access.query('media=="nv_api3"')['id'].values[0]
sc_3 = access.query('media=="nv_api3"')['secret'].values[0]


In [41]:
import pandas as pd
import urllib.request
import urllib.parse
import json
import time
import random
from tqdm import tqdm


# API 엑세스 정의
client_credentials = [
    {"client_id": id_1, "client_secret": sc_1},
    {"client_id": id_2, "client_secret": sc_2},
    {"client_id": id_3, "client_secret": sc_3},
]

def search_api(client_id, client_secret, query):
    encText = urllib.parse.quote(str(query)) 
    url = "https://openapi.naver.com/v1/search/shop?query=" + encText
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    
    try:
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read()
            return json.loads(response_body.decode('utf-8'))
        else:
            print("Error Code:", rescode)
            return rescode
    except urllib.error.HTTPError as e:
        print("HTTPError:", e.code)
        return e.code
    except urllib.error.URLError as e:
        print("URLError:", e.reason)
        return None
    except Exception as e:
        print("General Error:", str(e))
        return None

# 실패 시 지수로 대기 시간 설정    
def exponential_backoff(retries):
    delay = (2 ** retries) + random.uniform(0, 1)
    print(f"Exponential backoff: retry {retries}, sleeping for {delay:.2f} seconds")
    time.sleep(delay)

# 중간 저장
def save_intermediate_results(results, search_terms, start_index):
    result_df = pd.DataFrame.from_dict(results, orient='index', columns=['category1', 'category2', 'category3', 'category4'])
    result_df.reset_index(inplace=True)
    result_df.rename(columns={'index': 'name'}, inplace=True)
    result_df.to_parquet('intermediate_results.parquet')
    
    with open('remaining_search_terms.txt', 'w') as f:
        for term in search_terms[start_index:]:
            f.write(term + "\n")

# 런
current_credentials = 0
break_time = 60
results = {}
search_terms = list(set(df[product_col].tolist()))  # 타겟 키워드

for index, term in enumerate(tqdm(search_terms)):
    try:
        retries = 0
        while retries < 5:  # 실패 시 재시도 5회
            client_id = client_credentials[current_credentials]["client_id"]
            client_secret = client_credentials[current_credentials]["client_secret"]
            
            result = search_api(client_id, client_secret, term)
            
            if result == 401:
                print("Unauthorized: Check your API credentials")
                current_credentials = (current_credentials + 1) % len(client_credentials)
                if current_credentials == 0:
                    print(f"API 소진, {break_time}초 대기 ")
                    time.sleep(break_time)
                retries += 1
            elif result == 429:
                current_credentials = (current_credentials + 1) % len(client_credentials)
                if current_credentials == 0:
                    print(f"API 소진, {break_time}초 대기 ")
                    time.sleep(break_time)
                retries += 1
                exponential_backoff(retries)
            else:
                if isinstance(result, dict):
                    items = result.get('items', [])
                    if len(items) > 0:
                        ct1 = items[0].get('category1', None)
                        ct2 = items[0].get('category2', None)
                        ct3 = items[0].get('category3', None)
                        ct4 = items[0].get('category4', None)
                        results[term] = [ct1, ct2, ct3, ct4]
                    else:
                        results[term] = [None, None, None, None]
                else:
                    results[term] = [None, None, None, None]
                break
            
            time.sleep(1)  # 슬립
    except Exception as e:
        print(f"An error occurred: {e}")
        save_intermediate_results(results, search_terms, index)
        raise

# 최종 결과 저장
result_df = pd.DataFrame.from_dict(results, orient='index', columns=['category1', 'category2', 'category3', 'category4'])
result_df.reset_index(inplace=True)
result_df.rename(columns={'index': 'name'}, inplace=True)

result_df.to_parquet('final_results.parquet')



100%|██████████| 5/5 [00:02<00:00,  2.10it/s]


In [44]:
result_df

,productName_x,category1,category2,category3,category4
0,동서 옥수수차,식품,음료,차류,옥수수차
1,곰곰 통밀 식빵,식품,제과/제빵재료,기타제과/제빵재료,
2,프라이드 흰다리 새우살 (냉동),식품,수산물,해산물/어패류,새우
3,서울우유치즈 쭈욱 늘어나는 체다치즈,식품,유가공품,치즈,슬라이스치즈
4,한국야쿠르트 오리지널,식품,음료,우유/요거트,요거트


In [43]:
result_df.rename(columns={'name':product_col},inplace=True)

In [45]:
merged_df = df.merge(result_df,how='left',on=product_col)
merged_df

,Unnamed: 0,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName_x,quantity_x,gmv_x,commissionRate_x,commission_x,click_time,category1,category2,category3,category4
0,0,20240501,AF3155932,homecoin,061a7c2a-2d84-411f-9d0d-0ff82a51555a,460,7646299417,8084945774893014,1162842160,동서 옥수수차,1,14420.0,5.25,758.0,2024-05-01 22:39,식품,음료,차류,옥수수차
1,2,20240501,AF3155932,homecoin,d37c2db2-8c68-414f-a2c2-f1a684502778,400,1976267316,8814944474613677,1730791677,곰곰 통밀 식빵,1,3050.0,5.25,161.0,2024-04-30 21:40,식품,제과/제빵재료,기타제과/제빵재료,
2,3,20240501,AF3155932,homecoin,33e43dc5-9ae0-43b6-8761-3a8f60513792,460,7757208883,8814944174249659,6854375189,한국야쿠르트 오리지널,1,8800.0,5.25,462.0,2024-05-01 17:15,식품,음료,우유/요거트,요거트
3,4,20240501,AF3155932,homecoin,6413c766-a03c-4643-9f3d-2c4ef419180f,460,7869027269,8674940574517423,331808419,서울우유치즈 쭈욱 늘어나는 체다치즈,1,7180.0,5.25,377.0,2024-04-30 21:11,식품,유가공품,치즈,슬라이스치즈
4,5,20240501,AF3155932,homecoin,83aede64-3d61-4d08-8493-33ef07ce8d76,460,7750479802,8334947574225959,6429303041,프라이드 흰다리 새우살 (냉동),1,19000.0,5.25,998.0,2024-05-01 22:58,식품,수산물,해산물/어패류,새우


In [51]:
merged_df.to_csv("../result/inl_2024-06-14_2024-07-15_merged.csv",encoding='utf-8-sig')